In [ ]:
from fastai.tabular.all import *

In [ ]:
!pip install nbdev 
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder, StandardScaler
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from sklearn.tree import export_graphviz
import IPython, graphviz
import re
from nbdev.showdoc import *
import pandas as pd

In [ ]:
!pip install -Uqq waterfallcharts treeinterpreter dtreeviz

from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

from sklearn.inspection import plot_partial_dependence
from treeinterpreter import treeinterpreter
from waterfall_chart import plot as waterfall

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [ ]:
path = r'../input/revised-train-output/Revised Train Output.csv'
df_train = pd.read_csv(r'../input/revised-train-output/Revised Train Output.csv', skipinitialspace=True)
df_train.head()

In [ ]:
test_df = pd.read_csv("../input/revised-train-output/Revised Test Output.csv")
test_df.head()

In [ ]:
cat_names = [col for col in df_train if df_train[col].dtype!="float64"]
cont_names = [col for col in df_train if df_train[col].dtype=="float64"]
cont_names, cat_names

In [ ]:
cat_names.remove("target")
cat_names.remove("id")

In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df_train))

In [ ]:
splits

In [ ]:
procs = [Categorify,FillMissing,Normalize]

In [ ]:
to = TabularPandas(df_train, 
                   procs=procs, 
                   cat_names = cat_names, 
                   cont_names = cont_names, 
                   y_names='target', 
                   y_block=CategoryBlock, 
                   splits=splits)

In [ ]:
len(to.train),len(to.valid)

In [ ]:
to.xs.iloc[:2]

In [ ]:
dls = to.dataloaders(bs=2048)

In [ ]:
dls.show_batch()

In [ ]:
learn = tabular_learner(dls, opt_func= Adam,  metrics=accuracy, cbs=[ShowGraphCallback()])

In [ ]:
lr_min,lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))

In [ ]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [ ]:
learn.fit_one_cycle(20, 9.12e-03)

In [ ]:
# learn.recorder.plot_loss()

In [ ]:
learn.summary()


In [ ]:
row, clas, probs = learn.predict(df_train.iloc[0])

In [ ]:
# save_pickle(path/'to.pkl',to)

In [ ]:
row.show()

In [ ]:
test_df.head()


In [ ]:
test_df.drop("id", axis=1, inplace= True)


In [ ]:
test_df


In [ ]:
dl = learn.dls.test_dl(test_df)

In [ ]:
dl

In [ ]:
pred = learn.get_preds(dl=dl)

In [ ]:
pred

In [ ]:
preds = learn.get_preds(dl=dl)[0].argmax(1).numpy()
preds[:5]

In [ ]:
preds.shape

In [ ]:
sample = pd.read_csv("../input/tabular-playground-series-may-2022/sample_submission.csv")


In [ ]:
sample

In [ ]:
preds

In [ ]:
sub = pd.DataFrame({'id':sample.id, 'target': preds})
sub.to_csv('submission.csv', index=False)
sub.head()